In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from jinja2 import Template
from html2docx import html2docx

import ipywidgets as ipw
import datetime
import pandas as pd
import numpy as np

In [ ]:
from typing import List, Dict, Any, Optional, Tuple, Union
from dataclasses import dataclass, field
from enum import Enum
import datetime
from pathlib import Path

from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_ROW_HEIGHT_RULE
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.table import Table, _Cell
from docx.text.paragraph import Paragraph

import docx2pdf
import win32com.client
import os

In [ ]:
import research_generator

### Create Thesis Doc

In [ ]:
research_generator.create_thesis_doc()

### Create Linked NOTE Object

In [ ]:
from bloomberg_apis.rms_api import (
    initialize_notepublisher_client, CMTY, 
    initialize_rmsbql_client, 
    initialize_cdeuploader_client
    )
# from NotePublisher import NotePublisher

In [ ]:
# note_pub = NotePublisher(config_dir='config', environment='bbg')
note_pub = initialize_notepublisher_client()

In [ ]:
_tag_primary_ticker = note_pub.find_security('AAPL US Equity')
_tag_author = note_pub.find_user('Lucas Escobar', UserType.AUTHORS)
_tag_note_type = note_pub.find_taglist('BUYSIDE EQ: Note type', 'Meeting Note')

_cmty = note_pub.find_community('Lucas Test CMTY')

In [ ]:
note_pub.create_note(
    title='TEST',
    body='',
    as_of_date=int(datetime.datetime.strptime('2025-06-16', '%Y-%m-%d').timestamp() * 1000),
    tags = [_tag_primary_ticker, _tag_author, _tag_note_type],
    share_with = [_cmty],
    attachments= ['thesis.docx','thesis.pdf']
)

### Publish Assocatiated CDE Data

In [ ]:
uploader = initialize_cdeuploader_client()

In [ ]:
upload_frame = pd.DataFrame(columns=['parsekey','value','date','field'])
upload_frame

In [ ]:
req= {
    'parsekey' : 'SWIGGY IN Equity', 
    'field' : 'U1R2I', # UD_BQL_NAME_TEST
    'value' : 'Test_Value',
    'date' : '20250324'
}
upload_frame = pd.concat([upload_frame,pd.DataFrame([req])])
upload_frame

In [ ]:
# results = uploader.batch_upload_cde(
#     upload_frame,
#     max_workers=25,
#     batch_size=75,
#     batch_delay=1.0
# )

### Content Creation Imports

In [ ]:
from research_creation_system.orchestrator import ResearchContentOrchestrator
from research_creation_system.templates.thesis_template import (
    ThesisTemplate, 
    ThesisTemplateData,
    ThesisPublishingConfig,
    ThesisCDEMappingConfig
)
from research_creation_system.templates.light_update_template import (
    LightUpdateTemplate,
    LightUpdateTemplateData, 
    LightUpdatePublishingConfig,
    LightUpdateCDEMappingConfig
)
from research_creation_system.templates.company_model_template import (
    CompanyModelTemplate,
    CompanyModelTemplateData, 
    CompanyModelPublishingConfig,
    CompanyModelCDEMappingConfig
)
from research_creation_system.document_generation.excel_styling import ExcelStyleConfig
from research_creation_system.base.template_base import TaglistMapping
from research_creation_system.data_models import CompanyInfo, InvestmentRecommendation
from research_creation_system.utils import create_sample_thesis_data, create_sample_light_update_data, create_sample_company_model_template_data

In [ ]:
print("Testing Research Content System...")

# Initialize orchestrator
orchestrator = ResearchContentOrchestrator()

#### Thesis Template Sample

In [ ]:
print("Testing Thesis Template...")
thesis_data = create_sample_thesis_data()
thesis_config = ThesisPublishingConfig(
    community_name="Lucas Test CMTY",
    analyst_name="Lucas Escobar",
    taglists=[
        TaglistMapping("BUYSIDE EQ: Note type", "Meeting Note"),
        TaglistMapping("BUYSIDE EQ: Investment Theme", "Tech Trifecta")
    ]
)
thesis_cde_config = ThesisCDEMappingConfig()

try:
    thesis_results = orchestrator.publish_research(
        template_type=ThesisTemplate,
        ticker="AAPL US Equity",
        data=thesis_data,
        publishing_config=thesis_config,
        cde_config=thesis_cde_config,
        upload_cde=False
    )
    print("Thesis template executed successfully!")
except Exception as e:
    print(f"Thesis template failed: {e}")

#### Light Update Sample

In [ ]:
print("Testing Light Update Template...")
light_data = create_sample_light_update_data()
light_config = LightUpdatePublishingConfig(
    community_name="Lucas Test CMTY", 
    analyst_name="Lucas Escobar",
    taglists=[
        TaglistMapping("BUYSIDE EQ: Note type", "Meeting Note")
    ]
)
light_cde_config = LightUpdateCDEMappingConfig()

try:
    light_update_results = orchestrator.publish_research(
        template_type=LightUpdateTemplate,
        ticker="MSFT US Equity",
        data=light_data,
        publishing_config=light_config,
        cde_config=light_cde_config,
        upload_cde=False
    )
    print("Light update template executed successfully!")
except Exception as e:
    print(f"Light update template failed: {e}")

print("\nTesting complete!")


#### Company Model Sample

In [ ]:
print("Testing Company Model Template...")
model_data = create_sample_company_model_template_data()
model_config = CompanyModelPublishingConfig(
    community_name="Lucas Test CMTY",
    analyst_name="Lucas Escobar",
    taglists=[
        TaglistMapping("BUYSIDE EQ: Note type", "Company Model"),
        TaglistMapping("BUYSIDE EQ: Investment Theme", "Tech Trifecta")
    ]
)
model_cde_config = CompanyModelCDEMappingConfig()

try:
    model_results = orchestrator.publish_research(
        template_type=CompanyModelTemplate,
        ticker="CRM US Equity",
        data=model_data,
        publishing_config=model_config,
        cde_config=model_cde_config,
        style_config=ExcelStyleConfig,
        upload_cde=False
    )
    print("Company Model template executed successfully!")
except Exception as e:
    print(f"Company Model template failed: {e}")